In [15]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [39]:
import PyPDF2
import io
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [71]:
# Text extraction and removing un-necessary patterns and removal of introduction etc. functions
def extract_text_from_pdf(pdf_file_path):
    pdf_file = open(pdf_file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    extracted_text = ''
    for page in pdf_reader.pages:
        extracted_text += page.extract_text()

    return extracted_text


def remove_pattern(text, pattern):
    # Use regular expression to remove the specified pattern
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

def remove_text_before_pattern(text, pattern):
    start_index = text.find(pattern)
    if start_index != -1:
        cleaned_text = text[start_index:]
    else:
        cleaned_text = text

    return cleaned_text

def count_words(text):
    words = word_tokenize(text)
    return len(words)


In [41]:
extracted_text = extract_text_from_pdf("/content/crime-and-punishment.pdf")
# print(extracted_text)

pattern_to_remove = r'Free eBooks at Planet eBook\.com'
cleaned_text = remove_pattern(extracted_text, pattern_to_remove)
# print(cleaned_text)

pattern_to_find = "Part I Chapter I"
fcleaned_text = remove_text_before_pattern(cleaned_text, pattern_to_find)
# print(fcleaned_text)

total_words = count_words(fcleaned_text)

In [42]:
print(total_words)

258227


In [50]:
!pip install transformers

In [ ]:
!huggingface-cli login

In [ ]:
# Import the necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# model = AutoModelForSeq2SeqLM.from_pretrained("falcon-llm-7b")
# Specify the model name
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# As required pages are max 20(max 6000 words will come on 20 pages) so after
# text extraction count total no of words and set the desired words to 5900 and after that set the model max_length
# that is the output summary generation parameter to the value so that final summary is not exceeded to six pages.
N0 = total_words
N_desired = 5900
# t is no of times i want to pass the summary throgh model.
t = 3
r = 1 - (N_desired / N0) ** (1/t)

m_length = int(r*4000)

def summary_generator(text_chunk):
    text = """{text_chunk}"""
    # Generate the summary
    summary = model.generate(
        input_text=text,
        max_length=m_length,
        num_beams=5,
        temperature=0.8,
        return_tensors="pt",
    )
    return tokenizer.decode(summary[0], skip_special_tokens=True)
    # print(tokenizer.decode(summary[0], skip_special_tokens=True))

In [47]:
# funcition will split the text into chunks of 4000 so that model max token getting limit not exceeded.
def split_text_into_chunks(text, chunk_size):
    # Split the text into chunks of the specified size
    chunks = []
    words = text.split()

    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i+chunk_size])
        chunks.append(chunk)

    return chunks

# chunk_size = 4000


first time generates summary of full text, then send this summary again to model to further summarize it. repeat the procedure three times on updated summaries so that meaningfull and good summary generates and words also not exceed to 6000.

In [ ]:
# Split the text into chunks
text_chunks = split_text_into_chunks(fcleaned_text, 4000)

first_summary_text =""
for chunk in text_chunks:
    summary = summary_generator(chunk)
    first_summary_text+ = summary


In [ ]:
# Split the text into chunks
text_chunks = split_text_into_chunks(first_summary_text, 4000)

second_summary_text =""
for chunk in text_chunks:
    summary = summary_generator(chunk)
    second_summary_text+ = summary

In [ ]:
text_chunks = split_text_into_chunks(second_summary_text, 4000)

last_summary_text =""
for chunk in text_chunks:
    summary = summary_generator(chunk)
    last_summary_text+ = summary

In [ ]:
!pip install python-docx
!pip install reportlab

In [ ]:
# convert text to docs with alignment and justification. then docx to the pdf.
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN
from docx2pdf import convert
# Create a new DOCX document
doc = Document()
# Set the font style to Times New Roman and the font size to 20 points
font = doc.styles['Normal'].font
font.name = 'Times New Roman'
font.size = Pt(12)
paragraphs = last_summary_text.strip().split('\n\n')
# Add each paragraph to the document with justified alignment
for paragraph_text in paragraphs:
    p = doc.add_paragraph(paragraph_text.strip(), style='Normal')
    p.alignment = WD_ALIGN.JUSTIFY

# Save the document to a DOCX file
doc.save('final_summary.docx')
convert("/content/final_summary.docx")